<h1 align="center" style="color:#96d9f0;">Computational Intelligence for
Optimization - Project</h1>
<h3 align="center" style="color:#96d9f0;">Group AW - Wedding Seating Optimization</h3>

---

### <span style="color:#96d9f0;">Group Members</span>

<table>
  <thead style="color:#96d9f0;">
    <tr>
      <th>Name</th>
      <th>Email</th>
      <th>Student ID</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Afonso Dias</td>
      <td>20211540@novaims.unl.pt</td>
      <td>20211540</td>
    </tr>
    <tr>
      <td>Isabel Duarte</td>
      <td>20240545@novaims.unl.pt</td>
      <td>20240545</td>
    </tr>
    <tr>
      <td>Pedro Campino</td>
      <td>20240537@novaims.unl.pt</td>
      <td>20240537</td>
    </tr>
    <tr>
      <td>Rita Matos</td>
      <td>20211642@novaims.unl.pt</td>
      <td>20211642</td>
    </tr>
  </tbody>
</table>

In [1]:
#future improvements: test random fitnesses to set a baseline average to compare our results

In [2]:
#No swap mutation within the same table

---

### <span style="color:#96d9f0;">1. Imports</span>

This section includes all the necessary library imports required for the development of the project. We also use the library provided during the practical classes, as recommended, to support the implementation. Additionally, the relationship matrix used for evaluating guest compatibility is imported here.

In [3]:
import sys
sys.path.append('..')

In [4]:
import random
from copy import deepcopy
from library.solution import Solution
import csv
import os
import pandas as pd

In [5]:
# Import the relationship matrix

relationship_matrix = pd.read_csv('data/seating_data.csv')
relationship_matrix.drop(columns='idx', inplace=True)
relationship_matrix

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
0,0,5000,0,0,700,700,0,0,0,0,...,100,100,0,0,100,100,100,0,0,0
1,5000,0,700,700,0,0,300,300,500,500,...,100,100,0,100,0,0,0,0,0,0
2,0,700,0,2000,0,0,0,0,300,300,...,0,0,0,0,0,0,0,0,0,0
3,0,700,2000,0,0,0,900,400,300,300,...,0,0,0,0,0,0,0,0,0,0
4,700,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,100,0,0,2000,700,700
61,0,0,0,0,0,0,0,0,0,0,...,0,0,-1000,0,100,0,2000,0,700,700
62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,700,700,0,900


---

### <span style="color:#96d9f0;">2. Solution Representation</span>

In this section, we define the Wedding Sitting Solution class (`WSSolution`), which extends the base `Solution` class provided in the practical classes. This class represents a candidate solution for the wedding seating optimization problem.

Each solution consists of a list of tables, where each table is a list of guest indices. The class includes methods to generate a valid random initial representation and to evaluate the fitness of a solution based on the total relationship scores between guests seated at the same table.

In [6]:
class WSSolution(Solution):
    def __init__(self, repr=None, relationship_matrix=relationship_matrix, nr_of_tables=8):
        self.relationship_matrix = relationship_matrix
        self.nr_of_tables = nr_of_tables
        self.table_capacity = len(relationship_matrix) // nr_of_tables
        super().__init__(repr=repr)

    def random_initial_representation(self):
        tables = []
        left_idxs = [idx for idx in range(len(self.relationship_matrix))]
        for i in range(self.nr_of_tables):
            tables.append([])
            for j in range(self.table_capacity):
                idx = random.choice(left_idxs)
                left_idxs.remove(idx)
                # Check if idx is already in a table
                while any(idx in table for table in tables):
                    idx = random.randint(0, len(self.relationship_matrix) - 1)
                tables[i].append(idx)
        
        return tables
    
    def fitness(self):
        total_happiness = 0
        for i in range(len(self.repr)):
            table_happiness = 0

            # Calculate happiness for each table    
            for j in range(len(self.repr[i])-1):
                table_happiness += self.relationship_matrix.iloc[self.repr[i][j], self.repr[i][j + 1]]
            print(f"Table {i+1}: Fitness: {table_happiness}")
            total_happiness += table_happiness
            
        return total_happiness


In [9]:
solution = WSSolution()

print('Random solution:', solution)
print('Total Fitness:', solution.fitness())

Random solution: [[20, 48, 47, 40, 63, 31, 27, 39], [21, 58, 38, 62, 56, 55, 4, 14], [61, 0, 60, 46, 30, 16, 41, 22], [8, 13, 24, 12, 19, 17, 54, 1], [59, 11, 5, 2, 36, 52, 35, 29], [25, 3, 37, 23, 49, 18, 45, 34], [32, 43, 6, 42, 51, 10, 9, 57], [33, 28, 7, 50, 53, 26, 15, 44]]
Table 1: Fitness: 0
Table 2: Fitness: 100
Table 3: Fitness: 100
Table 4: Fitness: 400
Table 5: Fitness: 0
Table 6: Fitness: 0
Table 7: Fitness: 100
Table 8: Fitness: 500
Total Fitness: 1200
